In [41]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



pd.options.display.max_columns = None
pd.options.display.max_rows = 100


In [42]:
num_games = 4 

## MultiFeature RNN

In [54]:
season1920 = pd.read_csv(f'../Data/NewData/TimeSeriesMultFeaturesScaled(19-20)[{num_games}].csv')
season1819 =  pd.read_csv(f'../Data/NewData/TimeSeriesMultFeaturesScaled(18-19)[{num_games}].csv')

full_time_mult = season1819.append(season1920, ignore_index = True).sort_values(by = 'Date')
print(full_time_mult.shape)
full_time_mult.head(1)

(34336, 13)


,Name,Date,FDP,10,9,8,7,6,5,4,3,2,1
167,Kevon Looney,20181113.0,20.1,[0.18877551020408162 0.29846556886227543 0.118...,[0.1673469387755102 0.29890219560878245 0.1016...,[0.08775510204081632 0.2747005988023952 0.1186...,[0.16326530612244897 0.23241017964071856 0.084...,[0.22142857142857142 0.31424650698602796 0.135...,[0.17551020408163265 0.3201097804391218 0.1355...,[0.23775510204081632 0.25742265469061876 0.084...,[0.22448979591836735 0.20852045908183636 0.050...,[0.13877551020408163 0.1458957085828343 0.0169...,[0.26326530612244897 0.12630988023952097 0.016...


In [55]:
# def create_array(x): 
#     x = x[1:-1]
#     new_x = [float(i) for i in x.split(' ')]
#     return np.array(new_x)

In [56]:
min_max = MinMaxScaler()
standard = StandardScaler()
for i in range(1,11): 
    full_time_mult[f'{i}'] = full_time_mult[f'{i}'].map(create_array)

x = np.array(full_time_mult[[str(i) for i in list(range(1,11)[::-1])]]).squeeze().squeeze()
#x = min_max.fit_transform(x)
# y = np.array(full_time_mult[['FDP']])
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .8, random_state = 10)

In [58]:
print(x.shape)

(34336, 10)


In [39]:
x_train.shape, x_test.shape

((6867, 10), (27469, 10))

In [40]:
len(x_train[0][0])

14

In [36]:
assert False

AssertionError: 

### Single Feature (FDP)

In [ ]:
season1920 = pd.read_csv(f'../Data/NewData/TimeSeriesSingleFeatures(19-20)[{num_games}].csv')
season1819 = pd.read_csv(f'../Data/NewData/TimeSeriesSingleFeatures(18-19)[{num_games}].csv')

full_time = season1819.append(season1920, ignore_index = True).sort_values(by = 'Date')
full_time.head()

In [ ]:
full_time.shape

In [ ]:
min_max = MinMaxScaler()
standard = StandardScaler()
x = np.array(full_time[[str(i) for i in list(range(1,11)[::-1])]])
x = min_max.fit_transform(x)
y = np.array(full_time[['FDP']])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .8, random_state = 10)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
print(x_test.shape, x_train.shape)

In [ ]:
def multivar_RNN(x,y, metrics = ['mae']): 
    model = Sequential()
    units = 32
    drop = .3

#     #2nd type
#     model.add(LSTM(units, return_sequences = True, input_dim = x.shape[2], activation = 'relu'))
#     model.add(Dropout(drop))
#     model.add(LSTM(units = units, return_sequences = True, input_dim = x.shape[2], activation = 'relu'))
#     model.add(Dropout(drop))
#     model.add(LSTM(units = units, return_sequences = False, input_dim = x.shape[2], activation = 'relu'))
#     model.add(Dropout(drop))
#     model.add(Dense(1))#, activation = 'linear'))
    ADAM = Adam(0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)



    #2nd type
    model.add(LSTM(units, return_sequences = True, input_dim = x.shape[2],activation = 'relu'))
    model.add(Dropout(drop))
    model.add(LSTM(units = units, return_sequences = True, input_dim = x.shape[2], activation = 'relu'))
    model.add(Dropout(drop))
    model.add(LSTM(units = units, return_sequences = False, input_dim = x.shape[2], activation = 'relu'))
    model.add(Dropout(drop))
    model.add(Dense(1))
    
    model.compile(optimizer = 'adam', loss = 'mse', metrics = metrics)
    
    return model
        

In [ ]:
epochs = 100
batch_size = 16
rnn_model = multivar_RNN(x_train, y_train)
rnn_history = rnn_model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (x_test, y_test))